In [1]:
pip install streamlit folium pandas


  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/14/92/095c67f3d17b6116c2b1882bb5ac719939154ee5bc7e7610ee325159a101/streamlit-1.48.0-py3-none-any.whl.metadata
  Obtaining dependency information for altair!=5.4.0,!=5.4.1,<6,>=4.0 from https://files.pythonhosted.org/packages/aa/f3/0b6ced594e51cc95d8c1fc1640d3623770d01e4969d29c0bd09945fafefa/altair-5.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for blinker<2,>=1.5.0 from https://files.pythonhosted.org/packages/10/cb/f2ad4230dc2eb1a74edf38f1a38b9b52277f75bef262d8908e60d957e13c/blinker-1.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for cachetools<7,>=4.0 from https://files.pythonhosted.org/packages/00/f0/2ef431fe4141f5e334759d73e81120492b23b2824336883a91ac04ba710b/cachetools-6.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for protobuf<7,>=3.20 from https://files.pythonhosted.org/packages/6a/c9/b9689a2a250264a84e66c46d8862ba788ee7a641cdca3

In [2]:
import pandas as pd
import streamlit as st
import folium
from folium.plugins import MarkerCluster
from io import StringIO

st.set_page_config(page_title="Mapa Competencia Los Cabos", layout="wide")

# --- carga de datos ---
df = pd.read_csv("competencia_los_cabos.csv")

# normalizar campos de filtros
def split_tags(x):
    if not isinstance(x, str) or not x.strip():
        return []
    # separa por coma/;/| y limpia espacios
    raw = [t.strip() for t in x.replace("|", ",").replace(";", ",").split(",")]
    return [t for t in raw if t]

df["amenidades_tags"] = df["amenidades"].apply(split_tags)
for col in ["tipo_desarrollo", "diseno_estilo", "estado_desarrollo"]:
    df[col] = df[col].fillna("").astype(str).str.strip()

# opciones de filtros
amenities_all = sorted({a for lst in df["amenidades_tags"] for a in lst})
tipos_all      = sorted([t for t in df["tipo_desarrollo"].unique() if t])
estilos_all    = sorted([t for t in df["diseno_estilo"].unique() if t])
estados_all    = sorted([t for t in df["estado_desarrollo"].unique() if t])

# --- sidebar filtros ---
st.sidebar.header("Filtros")
sel_tipos   = st.sidebar.multiselect("Tipo de desarrollo", tipos_all)
sel_estilos = st.sidebar.multiselect("Estilo / diseño", estilos_all)
sel_estados = st.sidebar.multiselect("Estado del proyecto", estados_all)
sel_amen    = st.sidebar.multiselect("Amenidades", amenities_all)

col_btn1, col_btn2 = st.sidebar.columns(2)
reset = col_btn1.button("Reset filtros")
download_flag = col_btn2.button("Descargar filtrado")

if reset:
    sel_tipos.clear(); sel_estilos.clear(); sel_estados.clear(); sel_amen.clear()

# --- aplicar filtros ---
mask = pd.Series([True]*len(df))
if sel_tipos:
    mask &= df["tipo_desarrollo"].isin(sel_tipos)
if sel_estilos:
    mask &= df["diseno_estilo"].isin(sel_estilos)
if sel_estados:
    mask &= df["estado_desarrollo"].isin(sel_estados)
if sel_amen:
    # cada desarrollo debe contener TODAS las amenidades seleccionadas
    mask &= df["amenidades_tags"].apply(lambda L: all(a in L for a in sel_amen))

df_f = df[mask].copy()

# --- mapa ---
# centro: Santarena si existe; si no, promedio
if (df["nombre"].str.lower()=="santarena").any():
    centro = df.loc[df["nombre"].str.lower()=="santarena", ["lat","lon"]].iloc[0].astype(float).tolist()
else:
    centro = [df["lat"].astype(float).mean(), df["lon"].astype(float).mean()]

m = folium.Map(location=centro, zoom_start=14, tiles="OpenStreetMap")

# buffers (1km y 3km) sobre Santarena si existe
if (df["nombre"].str.lower()=="santarena").any():
    c = df.loc[df["nombre"].str.lower()=="santarena", ["lat","lon"]].iloc[0].astype(float).tolist()
    folium.Circle(location=c, radius=1000, fill=False).add_to(m)
    folium.Circle(location=c, radius=3000, fill=False).add_to(m)

cluster = MarkerCluster(name="Desarrollos").add_to(m)

def popup_html(row):
    parts = [f"<b>{row['nombre']}</b>"]
    if row["tipo_desarrollo"]: parts.append(row["tipo_desarrollo"])
    if row["tipologias_superficie_m2"]: parts.append(f"<b>Tipologías/m²:</b> {row['tipologias_superficie_m2']}")
    if row["amenidades"]: parts.append(f"<b>Amenidades:</b> {row['amenidades']}")
    if row["estado_desarrollo"]: parts.append(f"<b>Estado:</b> {row['estado_desarrollo']}")
    if row["website"]: parts.append(f"<a href='{row['website']}' target='_blank'>Sitio web</a>")
    return "<br>".join(parts)

for _, r in df_f.iterrows():
    try:
        lat, lon = float(r["lat"]), float(r["lon"])
    except:
        continue
    color = "red" if str(r["nombre"]).strip().lower()=="santarena" else "blue"
    folium.Marker(
        [lat, lon],
        tooltip=r["nombre"],
        popup=popup_html(r),
        icon=folium.Icon(color=color, icon="info-sign"),
    ).add_to(cluster)

folium.LayerControl(collapsed=False).add_to(m)

# render en Streamlit
st.markdown("### Mapa de competencia — Los Cabos")
st.caption(f"{len(df_f)} de {len(df)} desarrollos coinciden con los filtros.")
st.components.v1.html(m._repr_html_(), height=650, scrolling=False)

# descarga CSV filtrado
if download_flag:
    csv_buf = StringIO()
    df_f.drop(columns=["amenidades_tags"], errors="ignore").to_csv(csv_buf, index=False, encoding="utf-8-sig")
    st.download_button("Descargar CSV filtrado", data=csv_buf.getvalue(), file_name="competencia_filtrado.csv", mime="text/csv")


2025-08-11 22:58:26.026 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-11 22:58:26.039 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-11 22:58:26.150 
  command:

    streamlit run /Users/danib/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-08-11 22:58:26.150 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-11 22:58:26.151 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-11 22:58:26.151 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-11 22:58:26.152 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored 